# Tasca M10 T01

##### Install

In [1]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!pip install scrapy 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 995 kB 30.4 MB/s 
     |████████████████████████████████| 140 kB 49.7 MB/s 
     |████████████████████████████████| 384 kB 72.0 MB/s 
     |████████████████████████████████| 58 kB 6.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.12 which is incompatible.
Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
I

##### Import

In [2]:
# importing libraries

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.corpus import stopwords                            
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.probability import FreqDist



import requests
import csv
from bs4 import BeautifulSoup
from urllib.request import urlopen

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
#Crear Chrome Options - Necesario para Colab
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
browser = webdriver.Chrome('chromedriver',options=options)
from selenium.webdriver.common.keys import Keys

import unittest

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

import re

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')



from scrapy import Selector    

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


##### Function

In [3]:
def Beauty(url):
  page = requests.get(url)
  soup = BeautifulSoup(page.text, 'html.parser')
  return soup

def audioORvideo(s):
  n = 0
  last_word = s.split()[-1] 
  last_word

  if last_word == 'Video' or last_word == 'Audio':
    n = 1

  return n

## Exercici 1
### Realitza web scraping de dues de les tres pàgines web proposades utilitzant BeautifulSoup primer i Selenium després. 

- http://quotes.toscrape.com

- https://www.bolsamadrid.es

- www.wikipedia.es (fes alguna cerca primer i escrapeja algun contingut)

Definimos las páginas con las que vamos a trabajar como variables

In [4]:
url_quotes = 'http://quotes.toscrape.com'
url_wiki = 'https://en.wikipedia.org/wiki/Pappus_of_Alexandria'

#### Scraping with BeautifulSoup 

Empezamos con http://quotes.toscrape.com 

Ejecutamos la función que hemos creado que importará todo el documento HTML.

Verificamos leyendo el titulo que se ha importado correctamente.

In [5]:
soup_quotes = Beauty(url_quotes)
soup_quotes.title.string

'Quotes to Scrape'

Inspeccionando el código desde el navegador detectamos que el texto se encuentra dentro de la clase "quote".

    <div class="quote" itemscope itemtype="http://schema.org/CreativeWork">
        <span class="text" itemprop="text">“It is our choices, Harry, that show what we truly are, far more than our abilities.”</span>
        <span>by <small class="author" itemprop="author">J.K. Rowling</small>
        <a href="/author/J-K-Rowling">(about)</a>
        </span>
        <div class="tags">
            Tags:
            <meta class="keywords" itemprop="keywords" content="abilities,choices" /    > 
            
            <a class="tag" href="/tag/abilities/page/1/">abilities</a>
            
            <a class="tag" href="/tag/choices/page/1/">choices</a>
            
        </div>
    </div>

Confirmamos que hay 10 por página, al igual que número de cuotas en la página principal.

In [6]:
quote_elements = soup_quotes.find_all('div', class_='quote', itemscope="")
len(quote_elements)

10

Extraeremos dentro de cada clase "quote" el texto y el autor. Una vez extraido crearemos un dataset.

    <span class="text" itemprop="text">“It is our choices, Harry, that show what we truly are, far more than our abilities.”</span>
    <span>by <small class="author" itemprop="author">J.K. Rowling</small>

In [7]:
quote = []; author = [];

for x in quote_elements:
    quote.append(x.find('span', class_='text').string)
    author.append(x.find('small', class_='author').string)
    

In [8]:
quote_list = list(zip(quote, author))

df_quotes = pd.DataFrame({
    'Quote': quote,
    'Author': author
})

df_quotes.tail(2)

,Quote,Author
8,“A woman is like a tea bag; you never know how...,Eleanor Roosevelt
9,"“A day without sunshine is like, you know, nig...",Steve Martin


Continuamos con una entrada de la wikipedia repitiendo el mismo proceso.

In [9]:
soup_wiki = Beauty(url_wiki)
soup_wiki.title.string

'Pappus of Alexandria - Wikipedia'

En el caso de la wikipedia vamos a extraer todo el texto del artículo. Para ello debemos centrarnos en <p> que nos indica los parrafos del artículo.

In [10]:
text = ''
for paragraph in soup_wiki.find_all('p'):
    text += paragraph.text
    
text = re.sub(r'==.*?==+', '', text)
text = text.replace('\n', '')

text

'Pappus of Alexandria (/ˈpæpəs/; Greek: Πάππος ὁ Ἀλεξανδρεύς; c.\u2009 290\xa0– c.\u2009 350 AD) was one of the last great Greek mathematicians of antiquity known for his Synagoge (Συναγωγή) or Collection (c.\u2009 340),[1] and for Pappus\'s hexagon theorem in projective geometry. Nothing is known of his life, other than what can be found in his own writings: that he had a son named Hermodorus, and was a teacher in Alexandria.[2]Collection, his best-known work, is a compendium of mathematics in eight volumes, the bulk of which survives. It covers a wide range of topics, including geometry, recreational mathematics, doubling the cube, polygons and polyhedra.[1]Pappus was active in the 4th century AD. In a period of general stagnation in mathematical studies, he stands out as a remarkable exception.[3] "How far he was above his contemporaries, how little appreciated or understood by them, is shown by the absence of references to him in other Greek writers, and by the fact that his work h

Una vez tenemos el texto aprovecharemos los conocimientos del modulo anterior y pasamos la información a un dataset organizado por palabras y apariciones.

In [11]:
text = text.lower()
tokenized_word = word_tokenize(text)

tokenized_word = [w for w in tokenized_word if w.isalpha()]

fdist = FreqDist(tokenized_word)

word_df = pd.DataFrame(fdist.most_common(15))
word_df.columns = ['Common_words','count']
word_df.head()

,Common_words,count
0,the,149
1,of,136
2,and,66
3,a,46
4,in,40


#### Scraping with Selenium 

Extraeremos los mismos datos que anteriormente, de la página http://quotes.toscrape.com

Finalmente crearemos el dataset con los datos extraidos.


In [12]:
browser = webdriver.Chrome('chromedriver',options=options)
browser.get(url_quotes)

text = browser.find_elements(By.CLASS_NAME,'text')
name = browser.find_elements(By.CLASS_NAME,'author')

quote = []; author = [];

for value in text:
    quote.append(value.text)

for value in name:
    author.append(value.text)

browser.close()

quote_list = list(zip(quote, author))

df_quotes = pd.DataFrame({
    'Quote': quote,
    'Author': author
})

df_quotes.tail(2)

,Quote,Author
8,“A woman is like a tea bag; you never know how...,Eleanor Roosevelt
9,"“A day without sunshine is like, you know, nig...",Steve Martin


Extraeremos los mismos datos que anteriormente, de la página wikipedia y crearemos el dataset con los datos extraidos.

In [13]:
browser = webdriver.Chrome('chromedriver',options=options)
browser.get(url_wiki)

wiki = '' 



for i in range(1,24) :
  try: 
    text = browser.find_element("xpath", f'//*[@id="mw-content-text"]/div[1]/p[{i}]')
    wiki +=  text.text
  except : print('Error ' + i ) ; continue

wiki

'Pappus of Alexandria (/ˈpæpəs/; Greek: Πάππος ὁ Ἀλεξανδρεύς; c.\u2009 290 – c.\u2009 350 AD) was one of the last great Greek mathematicians of antiquity known for his Synagoge (Συναγωγή) or Collection (c.\u2009 340),[1] and for Pappus\'s hexagon theorem in projective geometry. Nothing is known of his life, other than what can be found in his own writings: that he had a son named Hermodorus, and was a teacher in Alexandria.[2]Collection, his best-known work, is a compendium of mathematics in eight volumes, the bulk of which survives. It covers a wide range of topics, including geometry, recreational mathematics, doubling the cube, polygons and polyhedra.[1]Pappus was active in the 4th century AD. In a period of general stagnation in mathematical studies, he stands out as a remarkable exception.[3] "How far he was above his contemporaries, how little appreciated or understood by them, is shown by the absence of references to him in other Greek writers, and by the fact that his work had 

In [14]:
text = wiki.lower()
tokenized_word = word_tokenize(text)

tokenized_word = [w for w in tokenized_word if w.isalpha()]

fdist = FreqDist(tokenized_word)

word_df = pd.DataFrame(fdist.most_common(15))
word_df.columns = ['Common_words','count']
word_df.head()

,Common_words,count
0,the,149
1,of,136
2,and,66
3,a,46
4,in,40


## Exercici 2
###  Documenta en un Word el teu conjunt de dades generat amb la informació que tenen els diferents arxius de Kaggle.

 Per saber més

A manera d'exemple del que es demana pots consultar aquest enllaç:

->https://www.kaggle.com/datasets/vivovinco/20212022-football-team-stats.



In [15]:
df_quotes.head()

,Quote,Author
0,“The world as we have created it is a process ...,Albert Einstein
1,"“It is our choices, Harry, that show what we t...",J.K. Rowling
2,“There are only two ways to live your life. On...,Albert Einstein
3,"“The person, be it gentleman or lady, who has ...",Jane Austen
4,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe


###**About Dataset df_quotes**
####Context

This dataset contains quotes from a scraped website.
These are all the quotes that appeared on the main page..

####Content

+10 rows and 2 columns.
Columns' description are listed below.

    Quote : Quotation
    Author : Person who made the quotation

####Acknowledgements

http://quotes.toscrape.*com*

In [16]:
word_df.head()

,Common_words,count
0,the,149
1,of,136
2,and,66
3,a,46
4,in,40


###**About Dataset word_df**
####Context

This dataset contains the 15 most common words on a wikipedia page from Pappus of Alexandria.

####Content

+15 rows and 2 columns.
Columns' description are listed below.

    Common_words : The specific word
    count : Number of repetitions

####Acknowledgements

https://en.wikipedia.org/wiki/Pappus_of_Alexandria

## Exercici 3
### Tria una pàgina web que tu vulguis i realitza web scraping mitjançant la llibreria Selenium primer i Scrapy després. 

In [17]:
url3 = 'https://www.bbc.com/sport'

##### Selenium



Extraeremos la información de BBC donde se encuentran los titulares.

In [18]:
browser = webdriver.Chrome('chromedriver',options=options)
browser.get(url3)


all_headlines_info = browser.find_elements(By.TAG_NAME, 'span')

Extraeremos todos los titulares y los pasamos a una lista.

In [19]:
head = []

for headlines in all_headlines_info:
    head.append(headlines.text)

 Eliminamos todos los titulares repetidos y toda aquella frase menor de 40 caracteres, dando por hecho que no son un titular.

Finalmente le damos algo de formato.

In [20]:
head = [x for x in head if len(x) >= 40]
head = list(set(head))

titulares = []

for i in head:
    i = re.sub("  ","",i)
    i = re.sub("\n","",i)
    titulares.append(i)

titulares


["'What a play!' Judge makes brilliant sliding catch for Yankees",
 "Qatar officials stop Peter Tatchell's LGBT protest",
 'Chequered flag podcast: United States Grand Prix review. AudioChequered flag podcast: United States Grand Prix review',
 'Alonso questions F1 direction after US GP penalty',
 'Rugby Union Weekly: Bring on the quarter-finals!',
 '5 Live Boxing: Katie Taylor, Delboy and the fight that no one wants. Audio5 Live Boxing: Katie Taylor, Delboy and the fight that no one wants',
 "Cristiano Ronaldo's behaviour 'unacceptable' - Micah Richards. Video",
 "Wrexham ban political message on striker's boots",
 "'What a play!' Judge makes brilliant sliding catch for Yankees. Video",
 'South Africa respond to Paine tampering allegation',
 'The women who fought for their own World Cup',
 'Lavorel dies from Isle of Man TT crash injuries',
 'Un-Australian? Hussey enjoying coaching England',
 'Pieters takes first steps 10 months on from crash',
 'LIVE. Champions League: Celtic & Shakht

##### Scrapy

Extraeremos la información de BBC donde se encuentran los titulares.

In [21]:
html = requests.get(url3).content
sel = Selector(text = html)

sports_titulares = sel.xpath('//p/span/text()').extract()

Extraeremos todos los titulares y le daremos algo de formato. Además quitaremos todos los titulares repetidos que acaban con 'Video' o 'Audio'.

In [22]:
titulares =[]
for i in sports_titulares:
  if audioORvideo(i) == 0:
    i = re.sub("  ","",i)
    i = re.sub("\n","",i)
    titulares.append(i)


titulares

['Champions League: Man City face Dortmund after Chelsea reach last 16',
 'Champions League: Celtic & Shakhtar Donetsk yet to find groove',
 'Stoinis smashes Australia to win over Sri Lanka',
 "'Top-ranked McIlroy perfectly placed for LIV fight'",
 'Chelsea reach knockout stage with win at Salzburg',
 'EFL: Two Championship matches plus League One & League Two',
 "Is Bayern's dominance bad for the Bundesliga?",
 'Ronaldo returns to Man Utd group training',
 'Pieters takes first steps 10 months on from crash',
 "'Emery has unfinished business' - but why now?",
 'Griner loses appeal against nine-year Russian jail term',
 'Lavorel dies from Isle of Man TT crash injuries',
 "'Doors are not open' for black managers - Vieira",
 "Wrexham ban political message on striker's boots",
 "'It’s getting ridiculous now' - O'Neil fury at VAR",
 'Murray battles back to beat Safiullin in Basel',
 'Roy back for England ODI series in Australia',
 "Qatar officials stop Peter Tatchell's LGBT protest",
 'McKe